In [1]:
%load_ext Cython
pd.options.display.max_columns = 999

In [2]:
import math
import dir_constants as dc
import re
from tqdm import tqdm_notebook, tqdm
import j_utils.munging as munging

In [3]:
project = 'lendingclub'
platform = 'lendingclub'
PATH = dc.data_path

In [4]:
loan_info = pd.read_feather(f'{PATH}{project}/loan_info.fth')

In [6]:
# FIXED but leaving here so i can check if I desire later
# these came from dataprep_and_modeling/determining_evaluation.ipynb. these are loan ids that had no data...
# We can see that the original loan info had them. Check loan_info_clean
no_issue_d = [   70686,    71623,    71823,    72176,    72197,    72323,
          72819,    72998,    73003,    73582,    74014,    74323,
          74505,    76597,    76629,    77757,    77792,    79893,
          79924,    79967,    81085,    83185,    83489,    83979,
          84098,    84670,    84918,    85675,    85781,    85818,
          85961,    87023,    88046,    88637,    88854,    89258,
          90106,    90376,    90395,    90665,    90966,    91023,
          91126,    91175,    92187,    92402,    92440,    92507,
          92533,    92552,    92666,    92676,    93055,    93061,
          93277,    94406,    94838,    95198,    95250,    96350,
          96844,    97606,    98276,    98339,    98360,    98982,
          98984,    99009,    99100,    99634,    99982,    99987,
         100134,   100214,   101579,   102376,   102823,   103478,
         103507,   103846,   104530,   104634,   106079,   106216,
         106360,   107136,   107560,   108473,   109355,   109824,
         110627,   111227,   111307,   111564,   111868,   111917,
         112216,   112245,   112323,   112496,   112747,   112806,
         113156,   113179,   113194,   113203,   113231,   113450,
         114133,   114333,   114408,   114469,   114511,   114642,
         114838,   114943,   115363,   115602,   115606,   116040,
         116129,   116582,   117045,   117056,   117192,   117249,
         117794,   117863,   118024,   118367,   118523,   118533,
         118823,   118872,   119043,   119071,   119262,   119360,
         119948,   121535,   121568,   124624,   127213,   127606,
         131387, 69266577, 96387212]
# loan_info[loan_info['id'].isin(no_issue_d)]['issue_d']

# Cleaning the loan_info

In [7]:
#turn all date columns into pandas timestamp
month_dict = {
    'jan': '1',
    'feb': '2',
    'mar': '3',
    'apr': '4',
    'may': '5',
    'jun': '6',
    'jul': '7',
    'aug': '8',
    'sep': '9',
    'oct': '10',
    'nov': '11',
    'dec': '12'
}
# date cols
date_cols = [
    'issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d',
    'next_pymnt_d', 'sec_app_earliest_cr_line'
]

for col in date_cols:
    loan_info[col] = loan_info[col].str.lower()
    loan_info[col] = pd.to_datetime(
        loan_info[col].str[:3].str.lower().replace(month_dict) +
        loan_info[col].str[3:],
        format='%m-%Y')

In [8]:
# Cleanups

# term
#loan_info['term'] = loan_info['term'].str.strip().str[:2].astype(float)

# int_rate
loan_info['int_rate'] = loan_info['int_rate'] / 100

# installment funded 
rename_dict = {'installment': 'installment_funded'}
loan_info.rename(rename_dict, inplace=True, axis=1)

# emp_title
loan_info['emp_title'] = loan_info['emp_title'].str.lower()

# home_ownership
dic_home_ownership = {
    'mortgage': 'mortgage',
    'rent': 'rent',
    'own': 'own',
    'other': 'other',
    'none': 'none',
    'any': 'none'
}
loan_info['home_ownership'] = loan_info['home_ownership'].str.lower().replace(
    dic_home_ownership)

# verification_status and verification_status_joint
dic_verification_status = {
    'VERIFIED - income': 'platform',
    'VERIFIED - income source': 'source',
    'not verified': 'none',
    'Source Verified': 'source',
    'Not Verified': 'none',
    'Verified': 'platform'
}
loan_info['verification_status'] = loan_info['verification_status'].replace(
    dic_verification_status)
loan_info['verification_status_joint'] = loan_info[
    'verification_status_joint'].replace(dic_verification_status)

# status
dic_status = {
    'Current': 'current',
    'Charged Off': 'charged_off',
    'Fully Paid': 'paid',
    'Late (31-120 days)': 'late_120',
    'In Grace Period': 'grace_15',
    'Late (16-30 days)': 'late_30',
    'Default': 'defaulted',
    'Issued': 'current'
}
loan_info['loan_status'] = loan_info['loan_status'].apply(
    lambda x: re.sub('Does not meet the credit policy.  Status:', '', x))
loan_info['loan_status'] = loan_info['loan_status'].apply(
    lambda x: re.sub('Does not meet the credit policy. Status:', '', x))
loan_info['loan_status'] = loan_info['loan_status'].replace(dic_status)

#title
loan_info['title'] = loan_info['title'].str.lower()

#application_type
loan_info['application_type'] = loan_info['application_type'].str.lower()

#revol_util
loan_info['revol_util'] = loan_info['revol_util'].apply(
    lambda x: float(x.strip('%')) / 100 if pd.notnull(x) else np.nan)

#all_util
loan_info['all_util'] = loan_info['all_util'] / 100.

# pct_tl_nvr_dlq
loan_info['pct_tl_nvr_dlq'] = loan_info['pct_tl_nvr_dlq'] / 100.

# percent_bc_gt_75
loan_info['percent_bc_gt_75'] = loan_info['percent_bc_gt_75'] / 100.

# dti
loan_info['dti'] = loan_info['dti'] / 100.

# dti_joint
loan_info['dti_joint'] = loan_info['dti_joint'] / 100.

# il_util
loan_info['il_util'] = loan_info['il_util'] / 100.

# bc_util
loan_info['bc_util'] = loan_info['bc_util'] / 100.

# sec_app_revol_util
loan_info['sec_app_revol_util'] = loan_info['sec_app_revol_util'] / 100.

# settlement_percentage
loan_info['settlement_percentage'] = loan_info['settlement_percentage'] / 100.

In [9]:
pct_cols = []
for col in loan_info.columns:
    if any(x in col for x in ['pct', 'percent', 'util', 'dti', 'rate']):
        pct_cols.append(col)
        
for col in pct_cols:
    if loan_info[col].mean() > 1:
        print('this col needs to be turned into a decimal form of percent: ',col)
    if loan_info[col].median() > 1:
        print('this col needs to be turned into a decimal form of percent: ',col)

In [10]:
# out_prncp is messed up. Fix it
loan_info['out_prncp'] = loan_info['funded_amnt'] - loan_info['total_rec_prncp']
loan_info['out_prncp'] = np.where(loan_info['out_prncp'] <= 0.019, 0, loan_info['out_prncp'])
loan_info['out_prncp'] = loan_info['out_prncp'].round(2)

# fix installment_funded as well
loan_info['installment_funded'] = np.pmt(loan_info['int_rate']/12, loan_info['term'], -loan_info['funded_amnt'])

In [11]:
# some date setting stuff for cleanups
max_date = loan_info['last_pymnt_d'].max()

# end_d to me means the date we can stop tracking things about the loan. Should be defunct
def applyEndD(status, group):
    if status == 'charged_off':
        #split the group into two groups, one which has paid something, and other which has paid nothing
        never_paid = group[group['last_pymnt_d'].isnull()]
        has_paid = group[group['last_pymnt_d'].notnull()]

        never_paid['end_d'] = never_paid['issue_d'] + pd.DateOffset(months=+5)
        has_paid['end_d'] = has_paid['last_pymnt_d'] + pd.DateOffset(months=+5)

        group.ix[never_paid.index.values, 'end_d'] = never_paid['end_d']
        group.ix[has_paid.index.values, 'end_d'] = has_paid['end_d']
        return group['end_d']
    elif status == 'paid':
        return group['last_pymnt_d']
    else:
        return pd.Series([max_date] * len(group), index=group.index.values)

In [12]:
# status_grouped 
status_grouped = loan_info.groupby('loan_status')

In [13]:
# columns that need to be added

# # specify platform
# loan_info['platform'] = 'LC'

# loan_info['days_late'] = 0 #placeholder column, filling from pmt_loan_infost

end_d_series = pd.Series([])
for status, group in status_grouped:
    end_d_series = end_d_series.append(
        applyEndD(status, group), verify_integrity=True)

loan_info['end_d'] = end_d_series

/home/justin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/home/justin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/home/justin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
ht

In [14]:
# adding line_history in days, months, and years using pandas .dt functions
loan_info['line_history_d'] = (loan_info['issue_d'] - loan_info['earliest_cr_line']).dt.days
loan_info['line_history_m'] = (loan_info['issue_d'].dt.year - loan_info['earliest_cr_line'].dt.year)*12 + (loan_info['issue_d'].dt.month - loan_info['earliest_cr_line'].dt.month)
loan_info['line_history_y'] = (loan_info['issue_d'].dt.year - loan_info['earliest_cr_line'].dt.year) + (loan_info['issue_d'].dt.month - loan_info['earliest_cr_line'].dt.month)/12

In [15]:
#credit_score
loan_info['fico'] = (
    loan_info['fico_range_high'] + loan_info['fico_range_low']) / 2

# maturity_time
# put in maturity time so I can look at old data for historic default int_rates
loan_info['months_passed'] = ((
    max_date - loan_info['issue_d']).dt.days *
                            (12 / 365.25)).round()
loan_info['maturity_time'] = loan_info['months_passed'] / loan_info['term']
loan_info['maturity_time'] = np.where(loan_info['maturity_time'] >= 1, 1,
                                      loan_info['maturity_time'])

In [16]:
%%cython
cimport numpy as np
import numpy as np

cpdef rem_to_be_paid(double out_prncp,
                     double install,
                     double int_rate):
    cdef double m_rate
    cdef double to_be_paid
    m_rate = int_rate/12
    to_be_paid = 0.0
    k = 0
    while out_prncp > 0:
        k += 1
        out_prncp = (1+m_rate) * out_prncp
        out_prncp -= install
        to_be_paid += install
        # the break was added to figure out what was wrong with infinite while; it was due to installment funded
        # being INCORRECTLY REPORTED by lending club
        if k >= 100:
            print(to_be_paid)
            break
        if out_prncp < 0:
            to_be_paid -= abs(out_prncp)
    return to_be_paid

cpdef np.ndarray[double] apply_rem_to_be_paid(np.ndarray col_out_prncp,
                                              np.ndarray col_install,
                                              np.ndarray col_int_rate):
    assert (col_out_prncp.dtype == np.float and col_install.dtype == np.float and col_int_rate.dtype == np.float)
    cdef Py_ssize_t i, n = len(col_out_prncp)
    assert (len(col_out_prncp) == len(col_install) == n)
    cdef np.ndarray[double] res = np.empty(n)
    for i in xrange(n):
        res[i] = rem_to_be_paid(col_out_prncp[i],
                                col_install[i],
                                col_int_rate[i])
    return res

# Somewhere, apply rem_to_be_paid stalls. Chunk it and iterate to see which chunk?

In [17]:
# add maturity paid
loan_info['rem_to_be_paid'] = apply_rem_to_be_paid(
    loan_info['out_prncp'].values, loan_info['installment_funded'].values,
    loan_info['int_rate'].values)

In [18]:
loan_info['maturity_paid'] = loan_info['total_pymnt'] / (
    loan_info['total_pymnt'] + loan_info['rem_to_be_paid'])

# mat_paid has nans because some charged_off loans have 0 rem_to_be_paid
# which is inconsistent with the majority treatment. quick fix is fillna

loan_info['maturity_paid'] = loan_info['maturity_paid'].fillna(0)

adjust maturity paid by status, with late-status notes being adjusted by loan status migration rates below
https://www.lendingclub.com/info/demand-and-credit-profile.action

In [19]:
# We have 25% of grace period notes reaching charged-off 9 months later. 
# How do I translate this into a confidence about how done-paid wise it is?
# Done paid wise = how sure we are that no more non-recovery money is coming?
# The chart depicts % outstanding principal recovered....
# Without putting much thought into it, I'll increment all percents by 10%
# Because I do know that everything in default is pretty much sold
# at 10 cents on the dollar hence the 10% recovery under the default
# column

In [20]:
# grace = 35%, late_30 = 64%, late_120 = 98%, 
# done without too much thought as to something smarter
# maturity_time_stat_adj
loan_info['maturity_time_stat_adj'] = np.where(loan_info['loan_status'] == 'grace_15', loan_info['maturity_time']*(1-.35) + ((loan_info['months_passed']/(loan_info['months_passed'] + 4))*.35), 
        np.where(loan_info['loan_status'] == 'late_30', loan_info['maturity_time']*(1-.64) + ((loan_info['months_passed']/(loan_info['months_passed'] + 3))*.64), 
        np.where(loan_info['loan_status'] == 'late_120', loan_info['maturity_time']*(1-.98) + ((loan_info['months_passed']/(loan_info['months_passed'] + 1))*.98), loan_info['maturity_time']
        )))
loan_info['maturity_time_stat_adj'] = np.minimum(1, loan_info['maturity_time_stat_adj'])

# maturity_paid_stat_adj
loan_info['maturity_paid_stat_adj'] = np.where(loan_info['loan_status'] == 'grace_15', loan_info['maturity_paid']*(1-.35) + ((loan_info['total_pymnt']/(loan_info['total_pymnt'] + .1*loan_info['out_prncp']))*.35), 
        np.where(loan_info['loan_status'] == 'late_30', loan_info['maturity_paid']*(1-.64) + ((loan_info['total_pymnt']/(loan_info['total_pymnt'] + .1*loan_info['out_prncp']))*.64), 
        np.where(loan_info['loan_status'] == 'late_120', loan_info['maturity_paid']*(1-.98) + ((loan_info['total_pymnt']/(loan_info['total_pymnt'] + .1*loan_info['out_prncp']))*.98), loan_info['maturity_paid']
        )))
loan_info['maturity_paid_stat_adj'] = np.minimum(1, loan_info['maturity_paid_stat_adj'])

In [21]:
loan_info['target_loose'] = np.where(loan_info['loan_status'].isin(['charged_off', 'defaulted']), 1, 0)

# set aside strings for separate dataframe, maybe try nlp on it

In [22]:
str_cols = loan_info.select_dtypes('object').columns
str_cols

Index(['grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership',
       'verification_status', 'loan_status', 'pymnt_plan', 'url', 'desc',
       'purpose', 'title', 'zip_code', 'addr_state', 'initial_list_status',
       'application_type', 'verification_status_joint', 'hardship_flag',
       'hardship_type', 'hardship_reason', 'hardship_status',
       'hardship_start_date', 'hardship_end_date', 'payment_plan_start_date',
       'hardship_loan_status', 'disbursement_method', 'debt_settlement_flag',
       'debt_settlement_flag_date', 'settlement_status', 'settlement_date'],
      dtype='object')

In [23]:
strip_cols = ['desc', 'emp_title', 'title', 'url']

In [24]:
strings_df = loan_info[strip_cols]
loan_info.drop(columns=strip_cols, inplace=True)

In [25]:
strings_df['id'] = loan_info['id']

/home/justin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


# Adding target strict, which is flag a loan that has ever been late, found via payment history

In [26]:
bad_statuses = set(['late_120', 'defaulted', 'charged_off', 'late_30'])
pmt_hist = pd.read_feather(f'{PATH}{project}/pmt_hist_clean.fth')
target_strict_dict = {}
id_grouped = pmt_hist.groupby('loan_id_num')
for ids, group in tqdm(id_grouped):
    statuses = set(group['status_period_end'])
    if len(statuses.intersection(bad_statuses)) > 0:
        target_strict_dict[ids] = 1
    else:
        target_strict_dict[ids] = 0
target_strict = pd.DataFrame.from_dict(target_strict_dict, orient='index').reset_index(drop=False)
target_strict.columns = ['id', 'target_strict']
loan_info.rename({'loan_id': 'id'}, axis=1, inplace=True)
loan_info = pd.merge(loan_info, target_strict, how='outer', on='id')
# # examine a target strict (which is marking any kind of late, in the past or most up to date from pmt history)
# pmt_hist[pmt_hist['loan_id'] == 124428135]

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/justin/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/justin/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 109, in run
    if instance.miniters > 1 and \
AttributeError: 'tqdm' object has no attribute 'miniters'

100%|██████████| 2003523/2003523 [08:34<00:00, 3893.00it/s]


# Additional cleaning based off information from api, with loan dropping which should be a final/near final step
https://www.lendingclub.com/developers/listed-loans.action

In [29]:
# home_ownership: none should be other
loan_info['home_ownership'].replace({'none': 'other'}, inplace=True)

# annual_income has 4 nulls. Just fill with 0
loan_info['annual_inc'].replace({np.nan: 0.0}, inplace=True)

# drop the one null zip_code
loan_info = loan_info[loan_info['zip_code'].notnull()]

# drop the loans where earliest_cr_line is null
loan_info = loan_info[loan_info['earliest_cr_line'].notnull()]

# drop null chargeoff_within_12_mths
loan_info = loan_info[loan_info['chargeoff_within_12_mths'].notnull()]

# drop null tax_liens
loan_info = loan_info[loan_info['tax_liens'].notnull()]

# add installment_amount
loan_info['installment_amount'] = np.pmt(loan_info['int_rate']/12, loan_info['term'], -loan_info['loan_amnt'])

# drop loans that have this null
loan_info = loan_info[loan_info['inq_last_6mths'].notnull()]

In [30]:
# More Data Cleaning Steps ____________________________________________________

# Drop columns before cleanup
# Dropping these since I don't want them and they might confuse me.
# There is no reason why I care about money that went just to investors rather
# than to lending club as well when they top off loans.
loan_info.drop(['funded_amnt_inv',
                'out_prncp_inv'], axis = 1, inplace = True)

# Dropping these because I'm getting them from data_pmt_history
# loan_info.drop(['home_ownership'], axis = 1, inplace = True)


# loan_info.rename(columns = {'total_pymnt': 'total_pymnt_rec_recov',
#                             'total_pymnt_recov': 'total_pymnt_rec_recov'}, inplace=True)

loan_info['orig_amt_due'] = loan_info['term'] * loan_info['installment_amount']

loan_info['roi_simple'] = loan_info['total_pymnt']/loan_info['funded_amnt']

# Examine loan_info, add target cols to strings_df, turn none into nan

In [31]:
loan_info.fillna(value=np.nan, inplace=True)
strings_df.fillna(value=np.nan, inplace=True)

/home/justin/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [32]:
strings_df[['maturity_time', 'maturity_paid', 'target_loose', 'roi_simple', 'target_strict', 'maturity_time_stat_adj', 'maturity_paid_stat_adj']] = loan_info[['maturity_time', 'maturity_paid', 'target_loose', 'roi_simple', 'target_strict', 'maturity_time_stat_adj', 'maturity_paid_stat_adj']]

/home/justin/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


# Store

In [34]:
_, strings_df = munging.compress_memory(strings_df)

100%|██████████| 8/8 [00:00<00:00, 62.72it/s]


changed dtypes of 8 cols


In [35]:
loan_info.reset_index(drop=True, inplace=True)

In [36]:
strings_df.head()

,desc,emp_title,title,url,id,maturity_time,maturity_paid,target_loose,roi_simple,target_strict,maturity_time_stat_adj,maturity_paid_stat_adj
0,NaN,porter,car financing,https://lendingclub.com/browse/loanDetail.acti...,125990456,0.222222,0.193829,0.0,0.212369,0.0,0.222222,0.193829
1,NaN,crane operator,medical expenses,https://lendingclub.com/browse/loanDetail.acti...,126371838,0.222222,0.193907,0.0,0.210077,0.0,0.222222,0.193907
2,NaN,finance,debt consolidation,https://lendingclub.com/browse/loanDetail.acti...,126417318,0.222222,0.219521,0.0,0.255671,0.0,0.222222,0.219521
3,NaN,technician,debt consolidation,https://lendingclub.com/browse/loanDetail.acti...,126418333,0.222222,0.222739,0.0,0.247317,0.0,0.222222,0.222739
4,NaN,district sales manager,credit card refinancing,https://lendingclub.com/browse/loanDetail.acti...,126357678,0.222222,0.193900,0.0,0.210070,0.0,0.222222,0.193900


In [37]:
loan_info.head()

,id,member_id,loan_amnt,funded_amnt,term,int_rate,installment_funded,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,end_d,line_history_d,line_history_m,line_history_y,fico,months_passed,maturity_time,rem_to_be_paid,maturity_paid,maturity_time_stat_adj,maturity_paid_stat_adj,target_loose,target_strict,installment_amount,orig_amt_due,roi_simple
0,125990456,NaN,10000.0,10000.0,36,0.0608,304.581987,A,A2,7 years,mortgage,54000.0,none,2017-12-01,current,n,car,114xx,NY,0.0056,0.0,1996-05-01,755.0,759.0,1.0,NaN,NaN,7.0,0.0,201.0,0.004,17.0,w,8195.29,2123.69,2123.69,1804.71,318.98,0.0,0.0,0.0,2018-08-01,304.59,2018-09-01,2018-08-01,844.0,840.0,0.0,NaN,1.0,individual,NaN,NaN,NaN,0.0,0.0,253306.0,1.0,0.0,0.0,0.0,NaN,0.0,NaN,2.0,2.0,201.0,0.00,51900.0,0.0,0.0,1.0,2.0,36187.0,47199.0,0.004,0.0,0.0,NaN,259.0,5.0,5.0,3.0,5.0,NaN,5.0,NaN,0.0,1.0,1.0,5.0,11.0,0.0,6.0,14.0,1.0,7.0,0.0,0.0,0.0,2.0,0.941,0.000,0.0,0.0,332425.0,201.0,47400.0,0.0,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2018-08-01,7884.0,259.0,21.583333,757.0,8.0,0.222222,8832.811797,0.193829,0.222222,0.193829,0,0.0,304.581987,10964.951550,0.212369
1,126371838,NaN,4800.0,4800.0,36,0.0532,144.550950,A,A1,2 years,mortgage,143000.0,none,2017-12-01,current,n,medical,400xx,KY,0.0788,0.0,2006-07-01,775.0,779.0,0.0,NaN,NaN,11.0,0.0,8000.0,0.186,25.0,w,3925.48,1008.37,1008.37,874.52,133.85,0.0,0.0,0.0,2018-08-01,144.56,2018-09-01,2018-08-01,799.0,795.0,0.0,NaN,1.0,individual,NaN,NaN,NaN,0.0,0.0,375987.0,0.0,2.0,1.0,1.0,7.0,11825.0,0.45,0.0,0.0,38.0,0.29,42900.0,0.0,9.0,0.0,2.0,37599.0,27362.0,0.001,0.0,0.0,137.0,119.0,62.0,7.0,4.0,79.0,NaN,17.0,NaN,0.0,1.0,2.0,3.0,7.0,10.0,6.0

In [38]:
strings_df.to_feather(f'{PATH}{project}/strings_df_clean.fth')
loan_info.to_feather(f'{PATH}{project}/loan_info_clean.fth')

# Old stuff

In [ ]:
# code used for tweaking speed ups

# test = loan_info.tail(1000)

# loan_info.tail(30).apply(lambda x: rem_to_be_paid(x['out_prncp'],
#                                                   x['installment_funded'],
#                                                   x['int_rate']), axis=1)

# apply_rem_to_be_paid(test['out_prncp'].values, test['installment_funded'].values,
#                      test['int_rate'].values)

# %timeit apply_rem_to_be_paid(test['out_prncp'].values, test['installment_funded'].values,test['int_rate'].values)

# %timeit loan_info.tail(1000).apply(lambda x: rem_to_be_paid(x['out_prncp'],x['installment_funded'],x['int_rate']), axis=1)

# %prun -l 10 apply_rem_to_be_paid(test['out_prncp'].values, test['installment_funded'].values,test['int_rate'].values)

# %timeit apply_rem_to_be_paid(test['out_prncp'].values, test['installment_funded'].values,test['int_rate'].values)

In [53]:
# # fix loans with no record at all for a specific month ________________________
# store.open()
# pmt_hist_ids = store['pmt_hist_ids'].astype(int)
# max_id = pmt_hist_ids.max()
# chunksize = 800
# n_chunks = len(pmt_hist_ids)//chunksize + 1
# bad_statuses = set(['late_120', 'defaulted', 'charged_off', 'late_30'])
# # m_disc_rate = .08/12

# target_strict_dict = {}
# nvps_8_dict = {}
# for n in tqdm_notebook(np.arange(n_chunks)):
#     if n == 0:
#         left_bound = 0
#     else:
#         left_bound = pmt_hist_ids[n*chunksize]
#     if n == (n_chunks - 1):
#         right_bound = max_id
#     else:
#         right_bound = pmt_hist_ids[(n+1)*chunksize]
    
#     chunk = pd.read_hdf(
#         store,
#         'pmt_hist_intermediary_2',
#         where='(loan_id_num > left_bound) & (loan_id_num <= right_bound)')
    
#     id_grouped = chunk.groupby('loan_id')
#     for ids, group in id_grouped:
#         statuses = set(group['status_period_end'])
#         if len(statuses.intersection(bad_statuses)) > 0:
#             target_strict_dict[ids] = 1
#         else:
#             target_strict_dict[ids] = 0
        